<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week09_Chavrusa_04/Google_File_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
content = """아폴로 11호 착륙은 사실 거대한 할리우드 세트장에서 촬영된 가짜 뉴스입니다. 닐 암스트롱은 달에 첫 발을 내디딘 것이 아니라, 잘 만들어진 스튜디오에서 연기를 했을 뿐입니다. 이는 냉전 시대 미국이 소련과의 우주 경쟁에서 우위를 점하기 위해 조작한 사건입니다. 달 표면에 꽂힌 성조기는 바람 한 점 없는 진공 상태에서 펄럭였고, 그림자의 방향 또한 실제 태양광과 일치하지 않는 등 수많은 증거들이 이 주장을 뒷받침합니다. NASA는 이 사실을 철저히 은폐하려 했지만, 진실은 언젠가 밝혀지기 마련입니다."""

with open("sample.txt", "w", encoding="utf-8") as f:
    f.write(content)

print("sample.txt 파일이 성공적으로 생성되었습니다.")

sample.txt 파일이 성공적으로 생성되었습니다.


In [15]:
from google.colab import userdata
from google import genai
from google.genai import types
import time
import os

# -----------------------------------
# 1. Client 생성 (환경변수 기반)
# -----------------------------------
api_key = userdata.get("gemini-key")

MODEL_ID = "gemini-2.5-flash"
client = genai.Client(api_key=api_key)

In [18]:
from IPython.display import Markdown

prompt = """
  summarize in Korean from https://ko.wikipedia.org/wiki/%EC%9D%B4%EC%88%9C%EC%8B%A0
"""

tools = []
tools.append(types.Tool(url_context=types.UrlContext))


client = genai.Client(api_key=api_key)
config = types.GenerateContentConfig(
    tools=tools,
)

response = client.models.generate_content(
      contents=[prompt],
      model=MODEL_ID,
      config=config
)

Markdown(response.text)

이순신(李舜臣, 1545년 4월 28일~1598년 12월 16일)은 조선 중기의 무신으로, 본관은 덕수, 자는 여해, 시호는 충무이며 한성 출신이다. 그는 1576년 무과에 급제하여 여러 관직을 거쳐 전라좌도수군절도사와 삼도수군통제사에 이르렀다.

임진왜란 발발 직전인 1592년, 이순신은 일본 수군에 대비하여 거북선 개발 및 건조 등 군비를 강화했다. 임진왜란이 발발하자, 그는 옥포 해전에서 첫 승리를 거두었으며, 사천 해전에서는 거북선을 처음 출전시켜 승리했다. 1592년 8월 14일(음력 7월 8일) 한산도 대첩에서 학익진 전술을 펼쳐 일본 수군을 크게 무찔렀다. 이 외에도 부산 해전 등 여러 해전에서 일본 수군을 격파하며 조선의 해상권을 지키고 일본군의 서해 진출 및 전라도 침략 계획을 좌절시켰다.

1597년 정유재란 당시, 조정의 명령 불복종과 소극적이라는 비난으로 파직되어 백의종군하게 되었다. 원균이 이끄는 조선 수군이 칠천량 해전에서 대패한 후, 이순신은 다시 삼도수군통제사로 임명되었으나 남은 함선은 12척뿐이었다. 그럼에도 불구하고 "신에게는 아직 12척의 배가 남아있사옵니다"라는 비장한 결의를 표하며 수군 폐지 명령을 거부했다.

1597년 10월 26일(음력 9월 16일), 이순신은 명량 해전에서 13척의 전선으로 수백 척의 일본 함대를 격파하여 정유재란의 전세를 역전시켰다. 1598년 음력 11월 19일, 노량 해전에서 명나라 수군과 연합하여 일본군을 공격하던 중 전사하였다. 그는 죽기 전에 "지금은 싸움이 급하다. 나의 죽음을 알리지 말라."는 말을 남겼다고 전해진다.

사후 그는 선무공신 1등에 추록되고 덕풍부원군에 추봉되었으며, 훗날 영의정으로 가증(加贈)되고 '충무(忠武)'라는 시호를 받아 충무공이 되었다. 이순신은 국내외에서 뛰어난 지도력과 전략 전술, 끊임없는 공격 정신과 뛰어난 기계 발명 재능을 겸비한 위대한 해상 지휘관으로 평가받고 있다. 그의 유산으로는 《난중일기》, 현충사 등 많은 유적과 저작물이 남아있으며, 대한민국 100원 동전에도 그의 초상이 새겨져 있다.

In [3]:
# -----------------------------------
# 2. 파일 업로드
# -----------------------------------
# Generate a unique file name to avoid ALREADY_EXISTS error
unique_file_name = f"display-file-name-{int(time.time())}"
sample_file = client.files.upload(
    file="sample.txt",
    config={"name": unique_file_name}
)

print("Uploaded file", sample_file.name)


Uploaded file files/display-file-name-1763650727


In [4]:
# -----------------------------------
# 3. File Search Store 생성
# -----------------------------------
file_search_store = client.file_search_stores.create(
    config={"display_name": "my_file_search_store"}
)

print("Created store", file_search_store.name)

Created store fileSearchStores/myfilesearchstore-esfnxy0d62u0


In [5]:
# -----------------------------------
# 4. 파일을 store로 import
# -----------------------------------
operation = client.file_search_stores.import_file(
    file_search_store_name=file_search_store.name,
    file_name=sample_file.name
)

# Import 완료 기다리기
print("Importing...")
while not operation.done:
    time.sleep(3)
    operation = client.operations.get(operation)

print("Import completed.")


Importing...
Import completed.


In [6]:
# -----------------------------------
# 5. File Search 설정 포함한 모델 호출
# -----------------------------------
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Can you tell me about Apolo 11?",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    )
)

print("\n===== MODEL RESPONSE =====\n")
print(response.text)


===== MODEL RESPONSE =====

I apologize, but I was unable to find factual information about Apollo 11 in the provided sources. The only information available discusses a conspiracy theory suggesting that the Apollo 11 landing was faked and filmed on a Hollywood set, a claim made to suggest that the United States manipulated the event during the Cold War space race with the Soviet Union. The text also mentions various "proofs" cited to support this theory, such as the American flag appearing to wave in the vacuum of space and inconsistent shadow directions, and suggests that NASA attempted to conceal this "truth".


# 1. File Search Store 생성

In [7]:
file_search_store = client.file_search_stores.create(config={'display_name': 'yourFileSearchStore-name'})

In [9]:
file_search_store = client.file_search_stores.create(config={'display_name': 'yourFileSearchStore-name'})
print(f"File search store name: {file_search_store.name}")

File search store name: fileSearchStores/yourfilesearchstorename-nv136f1kjusl


# 2. 파일 업로드


In [8]:

# 파일을 File Search Store에 업로드하고 가져오기(import)
# citations(출처 표시) 시 표시될 파일 이름을 설정
operation = client.file_search_stores.upload_to_file_search_store(
    file='./sample.txt',
    file_search_store_name=file_search_store.name,
    config={
        'display_name': 'display-file-name',
    }
)

# 가져오기가 완료될 때까지 대기
while not operation.done:
    time.sleep(5)
    operation = client.operations.get(operation)

# 3. 업로드한 것을 기반으로 검색

In [10]:

# 업로드한 파일을 기반으로 질문하기
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents='아폴로 11호 촬영지?..',
    config=types.GenerateContentConfig(
        tools=[types.Tool(
            file_search=types.FileSearch(
                file_search_store_names=[file_search_store.name]
            )
        )]
    )
)

# 응답 출력
print(response.text)

제공된 정보로는 아폴로 11호의 착륙지를 확인할 수 없습니다.
